In [6]:
!pip install pypdf pandas scikit-learn spacy sentence-transformers
!python -m spacy download en_core_web_sm


     -------------------------------------- 12.8/12.8 MB 808.9 kB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
!pip install tf-keras
!pip install keras==2.11.0


  Using cached keras-3.9.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0


  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.9.0
    Uninstalling keras-3.9.0:
      Successfully uninstalled keras-3.9.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires keras>=3.5.0, but you have keras 2.11.0 which is incompatible.


In [8]:
# Uninstall existing protobuf and reinstall a compatible version
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

# Restart the kernel after running this


Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "C:\Users\hp\anaconda3\lib\site-packages\pip\_internal\commands\uninstall.py", line 103, in run
    uninstall_pathset.commit()
  File "C:\Users\hp\anaconda3\lib\site-packages\pip\_internal\req\req_uninstall.py", line 424, in commit
    self._moved_paths.commit()
  File "C:\Users\hp\anaconda3\lib\site-packages\pip\_internal\req\req_uninstall.py", line 277, in commit
    save_dir.cleanup()
  File "C:\Users\hp\anaconda3\lib\site-packages\pip\_internal\utils\temp_dir.py", line 173, in cleanup
    rmtree(self._path)
  File "C:\Users\hp\anaconda3\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 328, in wrapped_f
    return self(f, *args, **kw)
  File "C:\Users\hp\anaconda3\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 408, in __call__
    do = self.iter(retry_state=ret

  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl (904 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires keras>=3.5.0, but you have keras 2.11.0 which is incompatible.


In [9]:
import streamlit as st
import PyPDF2
import spacy
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load NLP and BERT models
nlp = spacy.load("en_core_web_sm")
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to extract text from PDF
def extract_text_from_pdf(uploaded_file):
    reader = PyPDF2.PdfReader(uploaded_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Function to preprocess text
def preprocess_text(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Function to calculate similarity
def get_similarity_score(job_desc, resume_text):
    job_embedding = bert_model.encode([job_desc], convert_to_tensor=True)
    resume_embedding = bert_model.encode([resume_text], convert_to_tensor=True)
    similarity = cosine_similarity(job_embedding.cpu().detach().numpy(), 
                                   resume_embedding.cpu().detach().numpy())
    return round(similarity[0][0] * 100, 2)

# Streamlit UI
def run_streamlit():
    st.title("AI-Powered ATS Resume Screening System")
    st.write("Upload your resume and paste the job description to check ATS match score.")

    job_desc = st.text_area("Paste the Job Description Here", height=150)
    uploaded_file = st.file_uploader("Upload Your Resume (PDF)", type=["pdf"])

    if uploaded_file is not None and job_desc:
        with st.spinner("Processing..."):
            resume_text = extract_text_from_pdf(uploaded_file)
            cleaned_resume = preprocess_text(resume_text)
            cleaned_job_desc = preprocess_text(job_desc)
            score = get_similarity_score(cleaned_job_desc, cleaned_resume)

        st.success(f"✅ ATS Match Score: {score}%")
        st.write("📌 The higher the score, the better your resume matches the job description.")

# Check if running in a Jupyter Notebook
def is_notebook():
    try:
        get_ipython
        return True
    except NameError:
        return False

# Run in Jupyter Notebook
if is_notebook():
    print("Running in Jupyter Notebook mode...")
    
    # Test with sample text
    sample_job_desc = "Looking for a Data Scientist with experience in Python and Machine Learning."
    sample_resume_text = "I am a Data Scientist skilled in Python, Machine Learning, and Deep Learning."
    
    cleaned_job_desc = preprocess_text(sample_job_desc)
    cleaned_resume = preprocess_text(sample_resume_text)
    
    match_score = get_similarity_score(cleaned_job_desc, cleaned_resume)
    print(f"ATS Match Score (Sample Data): {match_score}%")
    
# Run Streamlit App
else:
    run_streamlit()


Running in Jupyter Notebook mode...
ATS Match Score (Sample Data): 66.32%
